Import the Operating System so we can access the files for the VTS library

Before we can run the Python code we need to download the latest zip file from the downloads and put the dll files into the libraries folder.

In [ ]:
import os
current_directory = os.getcwd()
publish_local = current_directory.replace("monte_carlo", "libraries\Vts.dll")

Use pip to install PythonNet Plotly and Numpy

In [ ]:
pip install pythonnet plotly numpy

Import the Core CLR runtime from PythonNet and add the reference for the VTS library and its dependencies

Import the namespaces from the Python libraries and the VTS library

In [ ]:
from pythonnet import set_runtime
set_runtime("coreclr")
import clr
clr.AddReference(publish_local) # Copy the VTS dlls into the libraries folders
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from Vts import *
from Vts.Common import *
from Vts.Extensions import *
from Vts.Modeling.Optimizers import *
from Vts.Modeling.ForwardSolvers import *
from Vts.SpectralMapping import *
from Vts.Factories import *
from Vts.MonteCarlo import *
from Vts.MonteCarlo.Sources import *
from Vts.MonteCarlo.Tissues import *
from Vts.MonteCarlo.Detectors import *
from Vts.MonteCarlo.Factories import *
from Vts.MonteCarlo.PhotonData import *
from Vts.MonteCarlo.PostProcessing import *
from System import Array

Setup the values for the simulations and plot the results using Plotly

ROfRho

In [ ]:
# create a SimulationInput object to define the simulation
detectorRange = DoubleRange(start=0, stop=40, number=201)
detectorInput = ROfRhoDetectorInput()
detectorInput.Rho = detectorRange
detectorInput.Name = "ROfRho"
detectors = Array.CreateInstance(IDetectorInput,1)
detectors[0] = detectorInput

simulationInput = SimulationInput()
simulationInput.N=1000
simulationInput.DetectorInputs= detectors

# create the simulation
simulation = MonteCarloSimulation(simulationInput)

# run the simulation
simulationOutput = simulation.Run()

# plot the results using Plotly
detectorResults = Array.CreateInstance(ROfRhoDetector,1)
detectorResults[0] = simulationOutput.ResultsDictionary["ROfRho"]
logReflectance = [r for r in detectorResults[0].Mean]
detectorMidpoints = [mp for mp in detectorRange.AsEnumerable()]

xLabel = "ρ [mm]"
yLabel = "log(R(ρ)) [mm-2]"

chart = go.Figure()
chart.add_trace(go.Scatter(x=detectorMidpoints, y=logReflectance, mode='markers'))
chart.update_layout( title="log(R(ρ)) [mm-2]", xaxis_title=xLabel, yaxis_title=yLabel)
chart.update_yaxes(type="log")
chart.show()


DAW vs CAW

In [ ]:
# Setup the detector input for the simulation
detectorRange = DoubleRange(start=0, stop=40, number=201)
detectorInput = ROfRhoDetectorInput()
detectorInput.Rho = detectorRange
detectorInput.Name = "ROfRho"
detectors = Array.CreateInstance(IDetectorInput,1)
detectors[0] = detectorInput

simulationOptions1 = SimulationOptions()
simulationOptions1.AbsorptionWeightingType = AbsorptionWeightingType.Discrete
# create a SimulationInput object to define the simulation
simulationInput1 = SimulationInput()
simulationInput1.N=1000
simulationInput1.OutputName = "MonteCarloROfRho-DAW"
simulationInput1.DetectorInputs= detectors
simulationInput1.Options = simulationOptions1

simulationOptions2 = SimulationOptions()
simulationOptions2.AbsorptionWeightingType = AbsorptionWeightingType.Continuous
# create a SimulationInput object to define the simulation
simulationInput2 = SimulationInput()
simulationInput2.N=1000
simulationInput2.OutputName = "MonteCarloROfRho-CAW"
simulationInput2.DetectorInputs = detectors
simulationInput2.Options = simulationOptions2

# create the simulations
simulation1 = MonteCarloSimulation(simulationInput1)
simulation2 = MonteCarloSimulation(simulationInput2)

# run the simulations
simulationOutput1 = simulation1.Run()
simulationOutput2 = simulation2.Run()

# plot the results using Plotly
detectorResults1 = Array.CreateInstance(ROfRhoDetector,1)
detectorResults1[0] = simulationOutput1.ResultsDictionary["ROfRho"]
logReflectance1 = [r for r in detectorResults1[0].Mean]
detectorMidpoints1 = [mp for mp in detectorRange.AsEnumerable()]

detectorResults2 = Array.CreateInstance(ROfRhoDetector,1)
detectorResults2[0] = simulationOutput2.ResultsDictionary["ROfRho"]
logReflectance2 = [r for r in detectorResults[0].Mean]
detectorMidpoints2 = [mp for mp in detectorRange.AsEnumerable()]

xLabel = "ρ [mm]"
yLabel = "log(R(ρ)) [mm-2]"

chart = go.Figure()
chart.add_trace(go.Scatter(x=detectorMidpoints1, y=logReflectance1, mode='markers', name='log(R(ρ)) [mm-2] - DAW'))
chart.add_trace(go.Scatter(x=detectorMidpoints2, y=logReflectance2, mode='markers', name='log(R(ρ)) [mm-2] - CAW'))
chart.update_layout( title="DAW vs CAW", xaxis_title=xLabel, yaxis_title=yLabel)
chart.update_yaxes(type="log")
chart.show()